In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available. Running on CPU-only mode.")

In [ ]:
import torch
print("CUDA dostępne:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Wykryta karta:", torch.cuda.get_device_name(0))


In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

# test


In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121



In [ ]:
import torch
print("CUDA dostępne:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Wykryta karta:", torch.cuda.get_device_name(0))

In [ ]:
import sys
print(sys.executable)

In [ ]:

%pip install ipykernel
%python -m ipykernel install --user --name cuda-env --display-name "Python (CUDA)"


In [ ]:
import torch
print(torch.__version__)
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Brak")



# trening

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model

# Train the model on GPU (device=0)
results = model.train(
    data='C:/Users/basia/PycharmProjects/proj1-nowotwor/model/data.yaml',
    epochs=100,
    imgsz=512,
    device=0  # GPU 0 = Twoja karta NVIDIA
)


# gotowy model

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("C:/Users/basia/OneDrive/Pulpit/now/runs/detect/train13/weights/best.pt")  # load a brain-tumor fine-tuned model

# Inference using the model
results = model.predict("C:/Users/basia/PycharmProjects/proj1-nowotwor/model/output3/test/images")

In [ ]:
results = pd.DataFrame(results)

In [ ]:
import os
import pandas as pd

# Ścieżki do folderów
images_folder = "C:/Users/basia/PycharmProjects/proj1-nowotwor/model/output3/test/images"
labels_folder = "C:/Users/basia/PycharmProjects/proj1-nowotwor/model/output3/test/labels"

# Lista do przechowywania danych
data = []

# Iteracja po plikach graficznych
for image_file in os.listdir(images_folder):
    if image_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        image_name = image_file
        label_file = os.path.splitext(image_file)[0] + ".txt"
        label_path = os.path.join(labels_folder, label_file)

        # Sprawdzenie czy odpowiadający plik tekstowy istnieje
        if os.path.isfile(label_path):
            with open(label_path, 'r', encoding='utf-8') as f:
                first_char = f.read(1)
                label = int(first_char) if first_char in ['0', '1'] else None
        else:
            label = None  # lub np. -1 jeśli chcesz oznaczyć brak pliku

        data.append({'image_name': image_name, 'label': label})

# Tworzenie DataFrame
test_real = pd.DataFrame(data)

# Wyświetlenie lub zapisanie wyników
print(df)
# df.to_csv("image_labels.csv", index=False)  # jeśli chcesz zapisać


In [ ]:
import os
import pandas as pd
import numpy as np

data = []

# Sprawdzamy, czy to pojedynczy obiekt Results, czy tablica (np. z predict())
results_array = results.values
results_flat = results_array.flatten() if isinstance(results_array, np.ndarray) else [results_array]

for result in results_flat:
    image_path = result.path
    image_name = os.path.basename(image_path)

    if result.boxes is not None and result.boxes.cls.numel() > 0:
        labels = result.boxes.cls.cpu().numpy().astype(int)
        label = int(pd.Series(labels).mode()[0])  # Najczęstszy label
    else:
        label = None  # Brak wykrycia

    data.append({'image_name': image_name, 'label': label})

# Tworzymy DataFrame
df = pd.DataFrame(data)
print(df)


In [ ]:
test_pred = df
test_pred = test_pred.fillna(0)

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score

# LEFT JOIN po image_name
df = test_real.merge(test_pred, on='image_name')

# Uzupełnienie brakujących predykcji na 0 (brak wykrycia = negatyw)
df['label_pred'] = df['label_pred'].fillna(0).astype(int)

# Obliczenie metryk
accuracy = accuracy_score(df['label_real'], df['label_pred'])
precision = precision_score(df['label_real'], df['label_pred'], zero_division=0)
recall = recall_score(df['label_real'], df['label_pred'], zero_division=0)

# Wyświetlenie wyników
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
